In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import os
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])

In [3]:
# Drop identified 'throw-aways'
application_df = application_df.drop(columns=['SPECIAL_CONSIDERATIONS', 'INCOME_AMT', 'STATUS'])

In [4]:
# How many buckets should be created from the application type column? 
app_type_counts = application_df['APPLICATION_TYPE'].value_counts()
print(f"{app_type_counts},\n")

# Determine which values to replace if counts are less than 1000?
replace_app_type = list(app_type_counts[app_type_counts < 1000].index)

# Replace in dataframe
for app in replace_app_type:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app, "Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64,



T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# How many buckets should be created from the classification column? 
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts.head(20)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
Name: CLASSIFICATION, dtype: int64

In [6]:
# Determine which values to replace if counts are less than 500?
replace_class = list(classification_counts[classification_counts < 500].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls, "Other")
    
# Check to make sure binning was successful
print(application_df.CLASSIFICATION.value_counts())
print(f'\n')
print(application_df.nunique())

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64


APPLICATION_TYPE       6
AFFILIATION            6
CLASSIFICATION         7
USE_CASE               5
ORGANIZATION           4
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64


In [7]:
# generate our categorical variable list
application_categories = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_categories

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION']

In [8]:
# Checking the number of unique values in each column
application_df[application_categories].nunique()

APPLICATION_TYPE    6
AFFILIATION         6
CLASSIFICATION      7
USE_CASE            5
ORGANIZATION        4
dtype: int64

In [9]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_categories]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_categories)
encode_df.head

/Users/leanna/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


<bound method NDFrame.head of        APPLICATION_TYPE_Other  APPLICATION_TYPE_T19  APPLICATION_TYPE_T3  \
0                         1.0                   0.0                  0.0   
1                         0.0                   0.0                  1.0   
2                         0.0                   0.0                  0.0   
3                         0.0                   0.0                  1.0   
4                         0.0                   0.0                  1.0   
...                       ...                   ...                  ...   
34294                     0.0                   0.0                  0.0   
34295                     0.0                   0.0                  0.0   
34296                     0.0                   0.0                  1.0   
34297                     0.0                   0.0                  0.0   
34298                     0.0                   0.0                  1.0   

       APPLICATION_TYPE_T4  APPLICATION_TYPE_T5  APPLICAT

In [10]:
# Merge one-hot encoded features 
application_df = application_df.merge(encode_df,left_index=True, right_index=True)

# Drop the original application categories
application_df = application_df.drop(columns=application_categories)
application_df.head()

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,...,CLASSIFICATION_Other,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust
0,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,6692,1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,142590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [11]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(columns=["IS_SUCCESSFUL"]).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
print(number_input_features)

hidden_nodes_layer1 =  6
hidden_nodes_layer2 = 5
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(
    tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

29
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 180       
                                                                 
 dense_1 (Dense)             (None, 5)                 35        
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 221
Trainable params: 221
Non-trainable params: 0
_________________________________________________________________


2022-04-04 14:24:31.120123: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# checkpoint_path:
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# callback: 
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=4000)

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6468 - accuracy: 0.6332
Epoch 2/100
804/804 [==============================] - 1s 974us/step - loss: 0.5926 - accuracy: 0.6885
Epoch 3/100
804/804 [==============================] - 1s 942us/step - loss: 0.5849 - accuracy: 0.7222
Epoch 4/100
804/804 [==============================] - 1s 935us/step - loss: 0.5813 - accuracy: 0.7235
Epoch 5/100
754/804 [===========================>..] - ETA: 0s - loss: 0.5783 - accuracy: 0.7249
Epoch 5: saving model to checkpoints/weights.05.hdf5
804/804 [==============================] - 1s 968us/step - loss: 0.5788 - accuracy: 0.7241
Epoch 6/100
804/804 [==============================] - 1s 932us/step - loss: 0.5770 - accuracy: 0.7247
Epoch 7/100
804/804 [==============================] - 1s 938us/step - loss: 0.5754 - accuracy: 0.7248
Epoch 8/100
804/804 [==============================] - 1s 955us/step - loss: 0.5716 - accuracy: 0.7254
Epoch 9/100
804/804 [===================

804/804 [==============================] - 1s 929us/step - loss: 0.5578 - accuracy: 0.7284
Epoch 65/100
505/804 [=================>............] - ETA: 0s - loss: 0.5557 - accuracy: 0.7298
Epoch 65: saving model to checkpoints/weights.65.hdf5
804/804 [==============================] - 1s 921us/step - loss: 0.5580 - accuracy: 0.7287
Epoch 66/100
804/804 [==============================] - 1s 904us/step - loss: 0.5578 - accuracy: 0.7283
Epoch 67/100
804/804 [==============================] - 1s 907us/step - loss: 0.5579 - accuracy: 0.7283
Epoch 68/100
804/804 [==============================] - 1s 909us/step - loss: 0.5578 - accuracy: 0.7283
Epoch 69/100
804/804 [==============================] - 1s 920us/step - loss: 0.5579 - accuracy: 0.7286
Epoch 70/100
499/804 [=================>............] - ETA: 0s - loss: 0.5561 - accuracy: 0.7295
Epoch 70: saving model to checkpoints/weights.70.hdf5
804/804 [==============================] - 1s 929us/step - loss: 0.5577 - accuracy: 0.7281
Epoch 7

In [13]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5596 - accuracy: 0.7240 - 323ms/epoch - 1ms/step
Loss: 0.559617817401886, Accuracy: 0.7239649891853333


In [14]:
# Attempt 1 results saved: 
nn.save("AlphabetSoupCharity_Optimization1.h5")

In [15]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(columns=["IS_SUCCESSFUL"]).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
print(number_input_features)

hidden_nodes_layer1 =  10
hidden_nodes_layer2 = 10
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(
    tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

29
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 10)                300       
                                                                 
 dense_4 (Dense)             (None, 10)                110       
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                                 
Total params: 421
Trainable params: 421
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# checkpoint_path:
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# callback: 
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=4000)

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6166 - accuracy: 0.6957
Epoch 2/100
804/804 [==============================] - 1s 965us/step - loss: 0.5737 - accuracy: 0.7231
Epoch 3/100
804/804 [==============================] - 1s 938us/step - loss: 0.5681 - accuracy: 0.7235
Epoch 4/100
804/804 [==============================] - 1s 951us/step - loss: 0.5664 - accuracy: 0.7238
Epoch 5/100
749/804 [==========================>...] - ETA: 0s - loss: 0.5650 - accuracy: 0.7256
Epoch 5: saving model to checkpoints/weights.05.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5651 - accuracy: 0.7254
Epoch 6/100
804/804 [==============================] - 1s 950us/step - loss: 0.5641 - accuracy: 0.7257
Epoch 7/100
804/804 [==============================] - 1s 977us/step - loss: 0.5641 - accuracy: 0.7253
Epoch 8/100
804/804 [==============================] - 1s 928us/step - loss: 0.5633 - accuracy: 0.7260
Epoch 9/100
804/804 [=====================

804/804 [==============================] - 1s 952us/step - loss: 0.5560 - accuracy: 0.7306
Epoch 65/100
488/804 [=================>............] - ETA: 0s - loss: 0.5548 - accuracy: 0.7321
Epoch 65: saving model to checkpoints/weights.65.hdf5
804/804 [==============================] - 1s 952us/step - loss: 0.5558 - accuracy: 0.7309
Epoch 66/100
804/804 [==============================] - 1s 920us/step - loss: 0.5559 - accuracy: 0.7310
Epoch 67/100
804/804 [==============================] - 1s 950us/step - loss: 0.5556 - accuracy: 0.7308
Epoch 68/100
804/804 [==============================] - 1s 931us/step - loss: 0.5562 - accuracy: 0.7304
Epoch 69/100
804/804 [==============================] - 1s 925us/step - loss: 0.5560 - accuracy: 0.7314
Epoch 70/100
495/804 [=================>............] - ETA: 0s - loss: 0.5507 - accuracy: 0.7343
Epoch 70: saving model to checkpoints/weights.70.hdf5
804/804 [==============================] - 1s 944us/step - loss: 0.5558 - accuracy: 0.7305
Epoch 7

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5576 - accuracy: 0.7243 - 327ms/epoch - 1ms/step
Loss: 0.557583212852478, Accuracy: 0.7243148684501648


In [18]:
# Attempt 2 results saved: 
nn.save("AlphabetSoupCharity_Optimization2.h5")

In [19]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(columns=["IS_SUCCESSFUL"]).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
print(number_input_features)

hidden_nodes_layer1 =  10
hidden_nodes_layer2 = 10
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh"))

# Second hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Output layer
nn.add(
    tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

29
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 10)                300       
                                                                 
 dense_7 (Dense)             (None, 10)                110       
                                                                 
 dense_8 (Dense)             (None, 1)                 11        
                                                                 
Total params: 421
Trainable params: 421
Non-trainable params: 0
_________________________________________________________________


In [20]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# checkpoint_path:
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# callback: 
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=4000)

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5935 - accuracy: 0.7025
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5703 - accuracy: 0.7242
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5669 - accuracy: 0.7268
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5645 - accuracy: 0.7272
Epoch 5/100
752/804 [===========================>..] - ETA: 0s - loss: 0.5636 - accuracy: 0.7259
Epoch 5: saving model to checkpoints/weights.05.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5632 - accuracy: 0.7267
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5627 - accuracy: 0.7270
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5620 - accuracy: 0.7283
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5613 - accuracy: 0.7275
Epoch 9/100
804/804 [==============================] -

804/804 [==============================] - 1s 953us/step - loss: 0.5543 - accuracy: 0.7322
Epoch 65/100
539/804 [===================>..........] - ETA: 0s - loss: 0.5564 - accuracy: 0.7280
Epoch 65: saving model to checkpoints/weights.65.hdf5
804/804 [==============================] - 1s 956us/step - loss: 0.5540 - accuracy: 0.7315
Epoch 66/100
804/804 [==============================] - 1s 931us/step - loss: 0.5539 - accuracy: 0.7322
Epoch 67/100
804/804 [==============================] - 1s 936us/step - loss: 0.5545 - accuracy: 0.7313
Epoch 68/100
804/804 [==============================] - 1s 932us/step - loss: 0.5540 - accuracy: 0.7324
Epoch 69/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5539 - accuracy: 0.7320
Epoch 70/100
488/804 [=================>............] - ETA: 0s - loss: 0.5532 - accuracy: 0.7332
Epoch 70: saving model to checkpoints/weights.70.hdf5
804/804 [==============================] - 1s 973us/step - loss: 0.5540 - accuracy: 0.7325
Epoch 71/

In [21]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5568 - accuracy: 0.7269 - 328ms/epoch - 1ms/step
Loss: 0.5568057894706726, Accuracy: 0.7268804907798767


In [22]:
# Attempt 3 results saved: 
nn.save("AlphabetSoupCharity_Optimization3.h5")